### Environment Setup

Python Kernel - Homebrew Python 3.9.20

In [ ]:
!pip3 install torch torchvision opencv-python-headless detectron2

In [ ]:
!pip3 install git+https://github.com/facebookresearch/detectron2.git

In [ ]:
!pip3 install pycocotools

In [14]:
import detectron2
import os
print(f'Detectron2 version: {detectron2.__version__}')
print(f'Detectron2 installation path: {detectron2.__file__}')

# Check for config files
cfg_dir = os.path.join(os.path.dirname(os.path.dirname(detectron2.__file__)), 'detectron2/model_zoo/configs')
print(f'Looking for configs in: {cfg_dir}')

# Check specifically for the Mask R-CNN config
mask_rcnn_config = os.path.join(cfg_dir, 'COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')
if os.path.exists(mask_rcnn_config):
    print(f'✅ Found Mask R-CNN config at: {mask_rcnn_config}')
else:
    print(f'❌ Mask R-CNN config not found at: {mask_rcnn_config}')
    
    # Try to find it elsewhere
    import glob
    configs = glob.glob(os.path.join(os.path.dirname(detectron2.__file__), '**', 'mask_rcnn_R_50_FPN_3x.yaml'), recursive=True)
    if configs:
        print(f'Found alternative config locations: {configs}')

Detectron2 version: 0.6
Detectron2 installation path: /Users/niti/Library/Python/3.9/lib/python/site-packages/detectron2/__init__.py
Looking for configs in: /Users/niti/Library/Python/3.9/lib/python/site-packages/detectron2/model_zoo/configs
✅ Found Mask R-CNN config at: /Users/niti/Library/Python/3.9/lib/python/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml


### Prepare The Dataset

Training Data

In [ ]:
import os
import shutil
from PIL import Image

# Define folder paths
folder_A = "/Users/niti/Downloads/yolo_annotations/train/images"
folder_B = "/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Filtered/Cell-Only"
folder_C = "/Users/niti/Desktop/macOS_D/sem8/CP303/FISH-Instance-Net/Cell-Dataset/masks"
folder_D = "/Users/niti/Desktop/macOS_D/sem8/CP303/FISH-Instance-Net/Cell-Dataset/images"


# Ensure output folders exist
os.makedirs(folder_C, exist_ok=True)
os.makedirs(folder_D, exist_ok=True)

# Get base names (without extension) and their corresponding extensions from folder A
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
files_A = {os.path.splitext(f)[0]: os.path.splitext(f)[1].lower() for f in os.listdir(folder_A) if os.path.splitext(f)[1].lower() in image_extensions}

# Copy images from A to C
for base_name, ext in files_A.items():
    shutil.copy(os.path.join(folder_A, base_name + ext), os.path.join(folder_C, base_name + ext))

# Copy matching base names from B to D and convert format
for file in os.listdir(folder_B):
    base_name, ext_B = os.path.splitext(file)
    if base_name in files_A:  # Check if the base name exists in folder A
        src_path_B = os.path.join(folder_B, file)
        dest_ext_A = files_A[base_name]  # Get the extension from Folder A
        dest_path_D = os.path.join(folder_D, base_name + dest_ext_A)

        # Convert the file to the format of Folder A
        with Image.open(src_path_B) as img:
            img = img.convert("RGB") if dest_ext_A in {".jpg", ".jpeg"} else img  # Ensure proper conversion
            img.save(dest_path_D, format=dest_ext_A.strip("."))  # Save in the required format

print("Files copied and converted successfully.")

Testing Data

In [10]:
import os
import shutil
from PIL import Image

# Define folder paths
folder_A = "/Users/niti/Downloads/yolo_annotations/valid/images"
folder_B = "/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Filtered/Cell-Only"
folder_C = "/Users/niti/Desktop/macOS_D/sem8/CP303/FISH-Instance-Net/Test-Dataset/masks"
folder_D = "/Users/niti/Desktop/macOS_D/sem8/CP303/FISH-Instance-Net/Test-Dataset/images"


# Ensure output folders exist
os.makedirs(folder_C, exist_ok=True)
os.makedirs(folder_D, exist_ok=True)

# Get base names (without extension) and their corresponding extensions from folder A
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
files_A = {os.path.splitext(f)[0]: os.path.splitext(f)[1].lower() for f in os.listdir(folder_A) if os.path.splitext(f)[1].lower() in image_extensions}

# Copy images from A to C
for base_name, ext in files_A.items():
    shutil.copy(os.path.join(folder_A, base_name + ext), os.path.join(folder_C, base_name + ext))

# Copy matching base names from B to D and convert format
for file in os.listdir(folder_B):
    base_name, ext_B = os.path.splitext(file)
    if base_name in files_A:  # Check if the base name exists in folder A
        src_path_B = os.path.join(folder_B, file)
        dest_ext_A = files_A[base_name]  # Get the extension from Folder A
        dest_path_D = os.path.join(folder_D, base_name + dest_ext_A)

        # Convert the file to the format of Folder A
        with Image.open(src_path_B) as img:
            img = img.convert("RGB") if dest_ext_A in {".jpg", ".jpeg"} else img  # Ensure proper conversion
            img.save(dest_path_D, format=dest_ext_A.strip("."))  # Save in the required format

print("Files copied and converted successfully.")

Files copied and converted successfully.


### Detectron2 Code

In [1]:
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
import random

def extract_instance_masks(image):
    """
    Extract instance masks from a color-coded segmentation image
    
    Args:
    image (numpy.ndarray): Color-coded segmentation image
    
    Returns:
    list: List of binary masks for each unique color (excluding black)
    """
    # Convert to uint8 if not already
    image = image.astype(np.uint8)
    
    # Get unique colors (excluding black)
    unique_colors = np.unique(image.reshape(-1, image.shape[2]), axis=0)
    unique_colors = [color for color in unique_colors if not np.array_equal(color, [0,0,0])]
    
    # Extract masks
    masks = []
    for color in unique_colors:
        # Create binary mask for this color
        mask = np.all(image == color, axis=-1)
        masks.append(mask)
    
    return masks

def get_cell_dicts(img_dir):
    """
    Parse cell segmentation dataset with color-coded masks
    
    Expected directory structure:
    cell_dataset/
    ├── images/
    │   ├── image1.png
    └── masks/
        ├── image1.png
    """
    dataset_dicts = []
    
    # Iterate through image files
    for img_filename in sorted(os.listdir(os.path.join(img_dir, 'images'))):
        if not img_filename.endswith(('.png', '.jpg', '.tif')):
            continue
        
        # Full paths
        img_path = os.path.join(img_dir, 'images', img_filename)
        mask_path = os.path.join(img_dir, 'masks', img_filename)
        
        # Read image and mask
        image = cv2.imread(img_path)
        mask_image = cv2.imread(mask_path)
        
        # Get image dimensions
        height, width = image.shape[:2]
        
        # Record for this image
        record = {
            "file_name": img_path,
            "image_id": img_filename,
            "height": height,
            "width": width,
            "annotations": []
        }
        
        # Extract instance masks
        masks = extract_instance_masks(mask_image)
        
        # Process each mask
        for mask in masks:
            # Find contours
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Get polygon representation
            polygon = contours[0].flatten().tolist()
            
            # Compute bounding box
            x, y, w, h = cv2.boundingRect(mask.astype(np.uint8))
            
            # Create annotation
            annotation = {
                "bbox": [x, y, x+w, y+h],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [polygon],
                "category_id": 0,  # Assuming single cell class
                "iscrowd": 0
            }
            
            record["annotations"].append(annotation)
        
        dataset_dicts.append(record)
    
    return dataset_dicts

# Rest of the code remains the same as in the previous notebook
def register_cell_dataset(dataset_dir):
    """Register the cell dataset with Detectron2"""
    # Training dataset
    DatasetCatalog.register("cell_train", lambda d=dataset_dir: get_cell_dicts(d))
    MetadataCatalog.get("cell_train").set(thing_classes=["cell"])
    
    # Validation dataset (if you have a separate validation set)
    DatasetCatalog.register("cell_val", lambda d=dataset_dir: get_cell_dicts(d))
    MetadataCatalog.get("cell_val").set(thing_classes=["cell"])

def setup_cfg(num_classes=1, output_dir='./output'):
    """Create configuration for Mask R-CNN"""
    cfg = get_cfg()
    cfg.merge_from_file("detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    
    # Modifications for your specific dataset
    cfg.DATASETS.TRAIN = ("cell_train",)
    cfg.DATASETS.TEST = ("cell_val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    
    # Pretrained weights
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
    
    # Number of classes
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    
    # Output directory
    cfg.OUTPUT_DIR = output_dir
    
    return cfg

# Visualization function
def visualize_dataset(dataset_dir):
    """
    Visualize the dataset to verify mask extraction
    """
    # Get dataset dictionary
    dataset_dicts = get_cell_dicts(dataset_dir)
    
    # Visualize a few samples
    for d in random.sample(dataset_dicts, min(3, len(dataset_dicts))):
        img = cv2.imread(d["file_name"])
        
        # Create visualizer
        visualizer = Visualizer(img[:, :, ::-1], 
                                metadata=MetadataCatalog.get("cell_train"), 
                                scale=1.2)
        
        # Draw annotations
        vis = visualizer.draw_dataset_dict(d)
        
        # Plot
        plt.figure(figsize=(12, 8))
        plt.imshow(vis.get_image())
        plt.axis('off')
        plt.title(f"Dataset Sample: {d['file_name']}")
        plt.show()
    

In [2]:
# Set dataset directory
dataset_directory2 = "/Users/vipulpatil/Desktop/FISH-Instance-Net/Cell-Dataset"

# Visualize dataset first to verify masks
# visualize_dataset(dataset_dir)

Config for GPU

In [3]:
def setup_cfg(num_classes=1, output_dir='./output'):
    """Create configuration for Mask R-CNN with correct config path"""
    from detectron2.config import get_cfg
    
    # Create configuration
    cfg = get_cfg()
    
    # Use the exact path you verified exists
    cfg_path = "/Users/niti/Library/Python/3.9/lib/python/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    cfg.merge_from_file(cfg_path)
    
    # Alternatively, use model_zoo which handles paths internally
    # from detectron2 import model_zoo
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    
    # Modifications for your specific dataset
    cfg.DATASETS.TRAIN = ("cell_train",)
    cfg.DATASETS.TEST = ("cell_val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    
    # Pretrained weights
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
    
    # Number of classes
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    
    # Output directory
    cfg.OUTPUT_DIR = output_dir
    
    # Training hyperparameters
    cfg.SOLVER.IMS_PER_BATCH = 4
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 1000
    cfg.SOLVER.STEPS = (700, 900)
    
    return cfg

Config for CPU

In [3]:
def setup_cfg(num_classes=1, output_dir='./output'):
    """Create configuration for Mask R-CNN with correct config path"""
    from detectron2.config import get_cfg
    
    # Create configuration
    cfg = get_cfg()
    
    # Use the exact path you verified exists
    cfg_path = "/opt/anaconda3/envs/detectron2-env/lib/python3.9/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    cfg.merge_from_file(cfg_path)
    
    # Alternatively, use model_zoo which handles paths internally
    # from detectron2 import model_zoo
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    
    # Force CPU
    cfg.MODEL.DEVICE = "cpu"
    
    # Modifications for your specific dataset
    cfg.DATASETS.TRAIN = ("cell_train",)
    cfg.DATASETS.TEST = ("cell_val",)
    cfg.DATALOADER.NUM_WORKERS = 2
    
    # Pretrained weights
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
    
    # Number of classes
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes
    
    # Output directory
    cfg.OUTPUT_DIR = output_dir
    
    # Training hyperparameters for CPU
    # Reduce batch size for CPU training
    cfg.SOLVER.IMS_PER_BATCH = 2
    # Lower learning rate for more stable CPU training
    cfg.SOLVER.BASE_LR = 0.0001
    # Reduce iterations for CPU (can be increased if needed)
    cfg.SOLVER.MAX_ITER = 500
    cfg.SOLVER.STEPS = (350, 450)
    
    return cfg

In [4]:
class CellTrainer(DefaultTrainer):
    """Custom trainer with COCO evaluation"""
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, output_dir=cfg.OUTPUT_DIR)

def train_instance_segmentation(dataset_dir):
    """
    Complete training pipeline for instance segmentation
    
    Args:
    dataset_dir (str): Path to the dataset directory
    """
    # 1. Register dataset
    register_cell_dataset(dataset_dir)
    
    # 2. Setup configuration
    cfg = setup_cfg(
        num_classes=1,  # Adjust if you have multiple cell types
        output_dir='./cell_segmentation_output'
    )
    
    # 3. Configure training hyperparameters for 500 iterations
    cfg.SOLVER.IMS_PER_BATCH = 4  # This is fine if your GPU can handle it
    cfg.SOLVER.BASE_LR = 0.0001  # Slightly higher learning rate for shorter training
    cfg.SOLVER.MAX_ITER = 500  # Your desired iteration count
    cfg.SOLVER.STEPS = (350, 450)  # Learning rate reduction at 70% and 90% of training
    cfg.SOLVER.GAMMA = 0.1  # Learning rate reduction factor
    cfg.SOLVER.WARMUP_FACTOR = 1.0 / 1000
    cfg.SOLVER.WARMUP_ITERS = 50  # Warm up for first 10% of training
    cfg.SOLVER.WEIGHT_DECAY = 0.0001  # Regularization to prevent overfitting
    
    # 4. Create output directory
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    # 5. Create trainer
    trainer = CellTrainer(cfg)
    
    # 6. Resume or start training
    trainer.resume_or_load(resume=False)
    
    # 7. Start training
    trainer.train()
    
    # 8. Run inference on test set
    # Optional: you can add test set evaluation here
    try:
        # Load the best model
        cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
        predictor = DefaultPredictor(cfg)
        
        # Evaluate on validation set
        evaluator = COCOEvaluator("cell_val", output_dir=cfg.OUTPUT_DIR)
        val_loader = build_detection_test_loader(cfg, "cell_val")
        inference_results = inference_on_dataset(predictor.model, val_loader, evaluator)
        print("Validation Results:", inference_results)
    
    except Exception as e:
        print(f"Inference evaluation failed: {e}")
    
    return cfg

In [5]:
# Optional: Uncomment to train
trained_cfg = train_instance_segmentation(dataset_directory2)

[05/11 19:19:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[05/11 19:21:09 d2.engine.train_loop]: Starting training from iteration 0


/opt/anaconda3/envs/detectron2-env/lib/python3.9/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/11 19:28:25 d2.utils.events]:  eta: 2:59:24  iter: 19  total_loss: 2.224  loss_cls: 0.7091  loss_box_reg: 0.6052  loss_mask: 0.6775  loss_rpn_cls: 0.1869  loss_rpn_loc: 0.02842    time: 21.9660  last_time: 27.0864  data_time: 0.1158  last_data_time: 0.0043   lr: 3.8062e-05  
[05/11 19:35:50 d2.utils.events]:  eta: 2:51:56  iter: 39  total_loss: 1.859  loss_cls: 0.443  loss_box_reg: 0.7077  loss_mask: 0.5706  loss_rpn_cls: 0.07468  loss_rpn_loc: 0.0394    time: 22.0904  last_time: 23.4827  data_time: 0.0178  last_data_time: 0.0780   lr: 7.8022e-05  
[05/11 19:43:13 d2.utils.events]:  eta: 2:46:10  iter: 59  total_loss: 1.494  loss_cls: 0.3471  loss_box_reg: 0.6427  loss_mask: 0.4364  loss_rpn_cls: 0.04792  loss_rpn_loc: 0.03718    time: 22.1195  last_time: 22.4970  data_time: 0.0282  last_data_time: 0.0138   lr: 0.0001  
[05/11 19:50:40 d2.utils.events]:  eta: 2:36:43  iter: 79  total_loss: 1.337  loss_cls: 0.3021  loss_box_reg: 0.6454  loss_mask: 0.3594  loss_rpn_cls: 0.0115  loss_

### Predict on New Images

Using trained_cfg

In [6]:
def predict_on_new_images(cfg, image_paths):
    """
    Run inference on new images
    
    Args:
        cfg: Detectron2 config
        image_paths: List of paths to images
    """
    # Load the trained model
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Confidence threshold
    predictor = DefaultPredictor(cfg)
    
    # Create output directory
    output_dir = os.path.join(cfg.OUTPUT_DIR, "new_predictions")
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each image
    import cv2
    from detectron2.utils.visualizer import Visualizer, ColorMode
    
    for i, image_path in enumerate(image_paths):
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Run inference
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize results
        v = Visualizer(
            img[:, :, ::-1],
            scale=1.0,
            instance_mode=ColorMode.SEGMENTATION
        )
        v = v.draw_instance_predictions(instances)
        result_img = v.get_image()[:, :, ::-1]
        
        # Save results
        output_path = os.path.join(output_dir, f"pred_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, result_img)
        
        print(f"Processed {image_path}: Found {len(instances)} cell instances")
        
        # Extract mask data if needed
        masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None
        scores = instances.scores.numpy() if instances.has("scores") else None
        
        # You can save masks or do further analysis here
    
    print(f"All predictions saved to {output_dir}")

# Example usage
# image_paths = ['/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Consolidated-Data/Cell-Only/AL 224_23.tif']
# predict_on_new_images(trained_cfg, image_paths)

In [ ]:
image_paths = ['/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/AL 276_24_KMT2A.png', '/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png']
predict_on_new_images(trained_cfg, image_paths)

[05/12 02:04:44 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./cell_segmentation_output/model_final.pth ...
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/AL 276_24_KMT2A.png: Found 19 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png: Found 6 cell instances
All predictions saved to ./cell_segmentation_output/new_predictions


Using Saved Model + Save Instance Masks


In [10]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import os
import cv2 as cv
import numpy as np

def load_cell_segmentation_model(model_path):
    """
    Load a trained cell segmentation model
    
    Args:
        model_path: Path to model_final.pth file
    
    Returns:
        cfg: Configuration
        predictor: Loaded model predictor
    """
    # Get a base config
    cfg = get_cfg()
    
    # Add your model's configuration
    # This part needs to match how you configured the training
    cfg_path = "/opt/anaconda3/envs/detectron2-env/lib/python3.9/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.DEVICE = "cpu"
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Set to match your original training
    
    # Set model path and inference threshold
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    
    # Create predictor
    predictor = DefaultPredictor(cfg)
    
    return cfg, predictor

def prediction_maker(predictor, image_paths):
    
    # Create output directory
    output_dir = os.path.join(cfg.OUTPUT_DIR, "new_predictions")
    os.makedirs(output_dir, exist_ok=True)
    combined_masks_dir = os.path.join(cfg.OUTPUT_DIR, "instance_masks_combined")
    os.makedirs(combined_masks_dir, exist_ok=True)
    masks_dir = os.path.join(cfg.OUTPUT_DIR, "instance_masks")
    os.makedirs(masks_dir, exist_ok=True)

    # Process each image
    import cv2
    from detectron2.utils.visualizer import Visualizer, ColorMode
    
    for i, image_path in enumerate(image_paths):
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Run inference
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize results
        v = Visualizer(
            img[:, :, ::-1],
            scale=1.0,
            instance_mode=ColorMode.SEGMENTATION
        )
        v = v.draw_instance_predictions(instances)
        result_img = v.get_image()[:, :, ::-1]
        
        # Save results
        output_path = os.path.join(output_dir, f"pred_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, result_img)
        
        print(f"Processed {image_path}: Found {len(instances)} cell instances")
        
        # Extract mask data if needed
        masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None
        scores = instances.scores.numpy() if instances.has("scores") else None

        if masks is not None:
            base_filename = os.path.splitext(os.path.basename(image_path))[0]
            num_instances = masks.shape[0]

            # Combined Mask Image
            if num_instances > 0:
                # Create a color-coded mask image where each instance has a different color
                height, width = masks[0].shape
                combined_mask = np.zeros((height, width, 3), dtype=np.uint8)
                
                # Generate unique colors for each instance
                colors = [
                    (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
                    for _ in range(num_instances)
                ]
                
                # Add each mask with its unique color
                for j, mask in enumerate(masks):
                    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
                    color_mask[mask] = colors[j]
                    combined_mask = cv.bitwise_or(combined_mask, color_mask)
                
                # Save combined mask
                combined_mask_path = os.path.join(combined_masks_dir, f"{base_filename}_all_instances.png")
                cv2.imwrite(combined_mask_path, combined_mask)
            
            # Single Masks Saved
            if num_instances > 0:
                current_masks_dir = os.path.join(masks_dir, base_filename)
                os.makedirs(current_masks_dir, exist_ok=True)

                for j, mask in enumerate(masks):
                    # Create a blank grayscale image
                    single_mask = np.zeros((height, width), dtype=np.uint8)
                    
                    # Set mask area to white (255)
                    single_mask[mask] = 255
                    
                    # Save the single instance mask
                    single_mask_path = os.path.join(current_masks_dir, f"{base_filename}_instance_{j+1}.png")
                    cv2.imwrite(single_mask_path, single_mask)
                    
        # You can save masks or do further analysis here
    
    print(f"All predictions saved to {output_dir}")
# Usage
model_path = "./cell_segmentation_output/model_final.pth"
cfg, predictor = load_cell_segmentation_model(model_path)
# image_paths = ['/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Consolidated-Data/Cell-Only/AL 224_23.tif']
# prediction_maker(predictor, image_paths)
# Now you can use predictor for inference

[05/12 09:55:52 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./cell_segmentation_output/model_final.pth ...


Generate Predictions on Test data

In [11]:
image_paths = []
test_images_dir = "/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images"
for filename in os.listdir(test_images_dir):
    if filename.endswith(('.png', '.jpg', '.tif')):
        image_paths.append(os.path.join(test_images_dir, filename))
prediction_maker(predictor, image_paths)

Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 108_24_CEN11ATM_XL.png: Found 14 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 195_24_5P9Q15Q_26_06_2024.png: Found 2 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png: Found 6 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MDS 601_23_8.png: Found 66 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 127_24_PLOIDY_06_05_2024.png: Found 4 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MDS 254_24_CEN8.png: Found 2 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 92_24_1Q1P_XL.png: Found 36 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 159_24_TP53_NF1.png: Found 5 cell instances
Processed /Use

### Evaluation on Test Data

In [8]:
import os
import json
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import random

def evaluate_on_test_data(trained_cfg, test_data_dir):
    """
    Evaluate trained model on test data
    
    Args:
        trained_cfg: Trained detectron2 config
        test_data_dir: Directory containing test data
        
    Returns:
        Dictionary of evaluation results
    """
    # 1. Register test dataset using your existing functions
    test_dataset_name = "cell_test"
    
    # First clear the dataset if it's already registered
    if test_dataset_name in DatasetCatalog:
        DatasetCatalog.remove(test_dataset_name)
    
    # Register using your function
    DatasetCatalog.register(test_dataset_name, lambda d=test_data_dir: get_cell_dicts(d))
    MetadataCatalog.get(test_dataset_name).set(thing_classes=["cell"])
    
    # 2. Configure model for inference
    trained_cfg.MODEL.WEIGHTS = os.path.join(trained_cfg.OUTPUT_DIR, "model_final.pth")
    trained_cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Lower threshold for evaluation
    predictor = DefaultPredictor(trained_cfg)
    
    # 3. Create output directory for evaluation results
    eval_output_dir = os.path.join(trained_cfg.OUTPUT_DIR, "test_evaluation")
    os.makedirs(eval_output_dir, exist_ok=True)
    
    # 4. Setup evaluator
    evaluator = COCOEvaluator(
        test_dataset_name, 
        output_dir=eval_output_dir,
        tasks=("bbox", "segm"),  # Evaluate both bounding boxes and segmentation
        allow_cached_coco=False  # Don't use cached results
    )
    
    # 5. Create data loader for test set
    test_loader = build_detection_test_loader(trained_cfg, test_dataset_name)
    
    # 6. Run evaluation
    print(f"Running evaluation on {test_dataset_name}...")
    eval_results = inference_on_dataset(
        predictor.model, 
        test_loader, 
        evaluator
    )
    
    # 7. Print detailed results
    print("\n=== Evaluation Results ===")
    for task, metrics in eval_results.items():
        print(f"\n{task} metrics:")
        for metric_name, value in metrics.items():
            print(f"  {metric_name}: {value}")
    
    # 8. Generate visualization of results on a few test images
    visualize_test_results(trained_cfg, test_dataset_name, eval_output_dir, num_images=5)
    
    # 9. Save results to file
    with open(os.path.join(eval_output_dir, "metrics.json"), "w") as f:
        json.dump(eval_results, f, indent=2)
    
    print(f"\nFull evaluation results saved to: {eval_output_dir}")
    
    return eval_results

def visualize_test_results(cfg, dataset_name, output_dir, num_images=5):
    """
    Visualize model predictions on test images
    
    Args:
        cfg: Trained config
        dataset_name: Test dataset name
        output_dir: Directory to save visualizations
        num_images: Number of images to visualize
    """
    # Setup predictor
    predictor = DefaultPredictor(cfg)
    
    # Get dataset
    dataset_dicts = DatasetCatalog.get(dataset_name)
    
    # Create visualization directory
    vis_dir = os.path.join(output_dir, "visualizations")
    os.makedirs(vis_dir, exist_ok=True)
    
    # Randomly select images
    random.seed(42)  # For reproducibility
    selected_images = random.sample(dataset_dicts, min(num_images, len(dataset_dicts)))
    
    # Process each selected image
    for i, d in enumerate(selected_images):
        # Read image
        img_path = d["file_name"]
        img = cv2.imread(img_path)
        
        # Get predictions
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize ground truth
        v_gt = Visualizer(img[:, :, ::-1], scale=1.0)
        v_gt = v_gt.draw_dataset_dict(d)
        gt_vis = v_gt.get_image()[:, :, ::-1]
        
        # Visualize predictions
        v_pred = Visualizer(img[:, :, ::-1], scale=1.0)
        v_pred = v_pred.draw_instance_predictions(instances)
        pred_vis = v_pred.get_image()[:, :, ::-1]
        
        # Create side-by-side comparison
        h, w = img.shape[:2]
        comparison = np.zeros((h, w*2, 3), dtype=np.uint8)
        comparison[:, :w, :] = gt_vis
        comparison[:, w:, :] = pred_vis
        
        # Add labels
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(comparison, "Ground Truth", (10, 30), font, 1, (0, 255, 0), 2)
        cv2.putText(comparison, "Prediction", (w + 10, 30), font, 1, (0, 255, 0), 2)
        
        # Save visualization
        base_name = os.path.basename(img_path)
        output_path = os.path.join(vis_dir, f"compare_{base_name}")
        cv2.imwrite(output_path, comparison)
        
        print(f"Processed image {i+1}/{num_images}: {base_name}")
        print(f"  Found {len(instances)} predictions")
        print(f"  Ground truth: {len(d['annotations'])} annotations")
    
    print(f"\nVisualizations saved to: {vis_dir}")

def analyze_result_statistics(eval_results):
    """
    Generate additional statistics and insights from evaluation results
    
    Args:
        eval_results: Dictionary of evaluation results
    """
    print("\n=== Additional Analysis ===")
    
    # For segmentation tasks
    if "segm" in eval_results:
        print("\nSegmentation Metrics:")
        segm_metrics = eval_results["segm"]
        
        # Overall performance
        ap = segm_metrics.get("AP", 0)
        print(f"  Mean AP: {ap:.3f}")
        
        # Performance by object size
        ap_small = segm_metrics.get("AP-small", 0)
        ap_medium = segm_metrics.get("AP-medium", 0)
        ap_large = segm_metrics.get("AP-large", 0)
        print(f"  AP by size: small={ap_small:.3f}, medium={ap_medium:.3f}, large={ap_large:.3f}")
        
        # Performance at different IoU thresholds
        ap50 = segm_metrics.get("AP50", 0)
        ap75 = segm_metrics.get("AP75", 0)
        print(f"  AP at IoU: AP50={ap50:.3f}, AP75={ap75:.3f}")
        
        # Model usage recommendation
        if ap < 0.3:
            print("\n  ⚠️ Model performance is low. Consider:")
            print("    - Training for more iterations")
            print("    - Using data augmentation")
            print("    - Checking dataset quality")
        elif ap < 0.5:
            print("\n  ⚠️ Model performance is moderate. May work for some applications but consider:")
            print("    - Fine-tuning hyperparameters")
            print("    - Using a larger or more diverse training set")
        else:
            print("\n  ✅ Model performance is good for cell segmentation tasks")
    
    # For detection tasks
    if "bbox" in eval_results:
        print("\nBounding Box Detection Metrics:")
        bbox_metrics = eval_results["bbox"]
        print(f"  Mean AP: {bbox_metrics.get('AP', 0):.3f}")
        print(f"  AP50: {bbox_metrics.get('AP50', 0):.3f}")

def calculate_per_image_metrics(cfg, test_dataset_name):
    """
    Calculate metrics on a per-image basis
    
    Args:
        cfg: Detectron2 config
        test_dataset_name: Name of test dataset
    
    Returns:
        Dictionary with per-image metrics
    """
    # Setup predictor
    predictor = DefaultPredictor(cfg)
    
    # Get dataset
    dataset_dicts = DatasetCatalog.get(test_dataset_name)
    
    # Results dictionary
    per_image_results = {}
    
    # Process each image
    for d in dataset_dicts:
        img_id = d["image_id"]
        
        # Read image
        img = cv2.imread(d["file_name"])
        
        # Get predictions
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Calculate metrics for this image
        gt_boxes = [anno["bbox"] for anno in d["annotations"]]
        pred_boxes = instances.pred_boxes.tensor.numpy() if len(instances) > 0 else []
        
        # Count predictions and ground truth
        num_gt = len(gt_boxes)
        num_pred = len(pred_boxes)
        
        # Simple statistics for this image
        per_image_results[img_id] = {
            "num_gt": num_gt,
            "num_pred": num_pred,
            "filename": os.path.basename(d["file_name"])
        }
    
    return per_image_results

def main(trained_cfg, test_data_dir):
    """
    Main function to evaluate model on test data
    
    Args:
        trained_cfg: Trained detectron2 config
        test_data_dir: Directory containing test data
    
    Returns:
        Dictionary of evaluation results
    """
    # Evaluate model on test data
    print(f"Evaluating model on test data from: {test_data_dir}")
    eval_results = evaluate_on_test_data(trained_cfg, test_data_dir)
    
    # Generate additional analysis
    analyze_result_statistics(eval_results)
    
    # Calculate per-image metrics
    per_image_metrics = calculate_per_image_metrics(trained_cfg, "cell_test")
    
    # Save per-image metrics
    with open(os.path.join(trained_cfg.OUTPUT_DIR, "test_evaluation", "per_image_metrics.json"), "w") as f:
        json.dump(per_image_metrics, f, indent=2)
    
    # Print summary of per-image metrics
    total_gt = sum(img["num_gt"] for img in per_image_metrics.values())
    total_pred = sum(img["num_pred"] for img in per_image_metrics.values())
    
    print("\n=== Per-Image Summary ===")
    print(f"Total images: {len(per_image_metrics)}")
    print(f"Total ground truth instances: {total_gt}")
    print(f"Total predicted instances: {total_pred}")
    print(f"Average instances per image: {total_gt / len(per_image_metrics):.2f}")
    print(f"Average predictions per image: {total_pred / len(per_image_metrics):.2f}")
    
    return eval_results

# Usage:
test_results = main(trained_cfg, "/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset")

Evaluating model on test data from: /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset
[05/12 02:05:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./cell_segmentation_output/model_final.pth ...
[05/12 02:05:48 d2.evaluation.coco_evaluation]: Trying to convert 'cell_test' to COCO format ...
[05/12 02:05:48 d2.data.datasets.coco]: Converting annotations of dataset 'cell_test' to COCO format ...)
[05/12 02:06:10 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[05/12 02:06:10 d2.data.datasets.coco]: Conversion finished, #images: 42, #annotations: 692
[05/12 02:06:10 d2.data.datasets.coco]: Caching COCO format annotations at './cell_segmentation_output/test_evaluation/cell_test_coco_format.json' ...
[05/12 02:06:33 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|    cell    | 692          |
|            |              |
[05/12 02:06:33 d2.data.dataset_mapper]:

### Predict on New Images

Using trained_cfg

In [ ]:
def predict_on_new_images(cfg, image_paths):
    """
    Run inference on new images
    
    Args:
        cfg: Detectron2 config
        image_paths: List of paths to images
    """
    # Load the trained model
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Confidence threshold
    predictor = DefaultPredictor(cfg)
    
    # Create output directory
    output_dir = os.path.join(cfg.OUTPUT_DIR, "new_predictions")
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each image
    import cv2
    from detectron2.utils.visualizer import Visualizer, ColorMode
    
    for i, image_path in enumerate(image_paths):
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Run inference
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize results
        v = Visualizer(
            img[:, :, ::-1],
            scale=1.0,
            instance_mode=ColorMode.SEGMENTATION
        )
        v = v.draw_instance_predictions(instances)
        result_img = v.get_image()[:, :, ::-1]
        
        # Save results
        output_path = os.path.join(output_dir, f"pred_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, result_img)
        
        print(f"Processed {image_path}: Found {len(instances)} cell instances")
        
        # Extract mask data if needed
        masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None
        scores = instances.scores.numpy() if instances.has("scores") else None
        
        # You can save masks or do further analysis here
    
    print(f"All predictions saved to {output_dir}")

# Example usage
# image_paths = ['/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Consolidated-Data/Cell-Only/AL 224_23.tif']
# predict_on_new_images(trained_cfg, image_paths)

In [ ]:
image_paths = ['/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/AL 276_24_KMT2A.png', '/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png']
predict_on_new_images(trained_cfg, image_paths)

[05/12 02:04:44 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./cell_segmentation_output/model_final.pth ...
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/AL 276_24_KMT2A.png: Found 19 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png: Found 6 cell instances
All predictions saved to ./cell_segmentation_output/new_predictions


Using Saved Model + Save Instance Masks


In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import os
import cv2 as cv
import numpy as np

def load_cell_segmentation_model(model_path):
    """
    Load a trained cell segmentation model
    
    Args:
        model_path: Path to model_final.pth file
    
    Returns:
        cfg: Configuration
        predictor: Loaded model predictor
    """
    # Get a base config
    cfg = get_cfg()
    
    # Add your model's configuration
    # This part needs to match how you configured the training
    cfg_path = "/opt/anaconda3/envs/detectron2-env/lib/python3.9/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.DEVICE = "cpu"
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Set to match your original training
    
    # Set model path and inference threshold
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    
    # Create predictor
    predictor = DefaultPredictor(cfg)
    
    return cfg, predictor

def prediction_maker(predictor, image_paths):
    
    # Create output directory
    output_dir = os.path.join(cfg.OUTPUT_DIR, "new_predictions")
    os.makedirs(output_dir, exist_ok=True)
    combined_masks_dir = os.path.join(cfg.OUTPUT_DIR, "instance_masks_combined")
    os.makedirs(combined_masks_dir, exist_ok=True)
    masks_dir = os.path.join(cfg.OUTPUT_DIR, "instance_masks")
    os.makedirs(masks_dir, exist_ok=True)

    # Process each image
    import cv2
    from detectron2.utils.visualizer import Visualizer, ColorMode
    
    for i, image_path in enumerate(image_paths):
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        # Run inference
        outputs = predictor(img)
        instances = outputs["instances"].to("cpu")
        
        # Visualize results
        v = Visualizer(
            img[:, :, ::-1],
            scale=1.0,
            instance_mode=ColorMode.SEGMENTATION
        )
        v = v.draw_instance_predictions(instances)
        result_img = v.get_image()[:, :, ::-1]
        
        # Save results
        output_path = os.path.join(output_dir, f"pred_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, result_img)
        
        print(f"Processed {image_path}: Found {len(instances)} cell instances")
        
        # Extract mask data if needed
        masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None
        scores = instances.scores.numpy() if instances.has("scores") else None

        if masks is not None:
            base_filename = os.path.splitext(os.path.basename(image_path))[0]
            num_instances = masks.shape[0]

            # Combined Mask Image
            if num_instances > 0:
                # Create a color-coded mask image where each instance has a different color
                height, width = masks[0].shape
                combined_mask = np.zeros((height, width, 3), dtype=np.uint8)
                
                # Generate unique colors for each instance
                colors = [
                    (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
                    for _ in range(num_instances)
                ]
                
                # Add each mask with its unique color
                for j, mask in enumerate(masks):
                    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
                    color_mask[mask] = colors[j]
                    combined_mask = cv.bitwise_or(combined_mask, color_mask)
                
                # Save combined mask
                combined_mask_path = os.path.join(combined_masks_dir, f"{base_filename}_all_instances.png")
                cv2.imwrite(combined_mask_path, combined_mask)
            
            # Single Masks Saved
            if num_instances > 0:
                current_masks_dir = os.path.join(masks_dir, base_filename)
                os.makedirs(current_masks_dir, exist_ok=True)

                for j, mask in enumerate(masks):
                    # Create a blank grayscale image
                    single_mask = np.zeros((height, width), dtype=np.uint8)
                    
                    # Set mask area to white (255)
                    single_mask[mask] = 255
                    
                    # Save the single instance mask
                    single_mask_path = os.path.join(current_masks_dir, f"{base_filename}_instance_{j+1}.png")
                    cv2.imwrite(single_mask_path, single_mask)
                    
        # You can save masks or do further analysis here
    
    print(f"All predictions saved to {output_dir}")
# Usage
model_path = "./cell_segmentation_output/model_final.pth"
cfg, predictor = load_cell_segmentation_model(model_path)
# image_paths = ['/Users/niti/Desktop/macOS_D/sem8/CP303/Compilation-Data-170325/FISH-All-Consolidated-Data/Cell-Only/AL 224_23.tif']
# prediction_maker(predictor, image_paths)
# Now you can use predictor for inference

[05/12 09:55:52 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./cell_segmentation_output/model_final.pth ...


In [ ]:
image_paths = []
test_images_dir = "/Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images"
for filename in os.listdir(test_images_dir):
    if filename.endswith(('.png', '.jpg', '.tif')):
        image_paths.append(os.path.join(test_images_dir, filename))
prediction_maker(predictor, image_paths)

Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 108_24_CEN11ATM_XL.png: Found 14 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 195_24_5P9Q15Q_26_06_2024.png: Found 2 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MBN 80_24_CCND1_BA_XL.png: Found 6 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MDS 601_23_8.png: Found 66 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 127_24_PLOIDY_06_05_2024.png: Found 4 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MDS 254_24_CEN8.png: Found 2 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 92_24_1Q1P_XL.png: Found 36 cell instances
Processed /Users/vipulpatil/Desktop/FISH-Instance-Net/Test-Dataset/images/MM 159_24_TP53_NF1.png: Found 5 cell instances
Processed /Use